<h1>Preparation of the German Twitter Data</h1>

<p>This file is for preparing the german tweets. The preparation is mainly for training my own model for sentiment detection with Flair. </p>

In [1]:
import csv
import numpy as np
import pandas as pd
import re
import nltk

<h1>Einlesen und bestimmung der Felder</h1>
<p>Durch die weitern Operatoren, kann ich die einzelnen Felder in DF bestimmen. Durch die Lambda Funktion werden die Str zu Listen, was die spätere verarbeitung erleichtert. 
    Durch die Parse_dates Funktion wird das Datumsfeld zu einem Timestamp, was die weiter Verarbeitung wesentlich leichter macht.</p>

In [226]:
df_manager_de = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Manager_DE/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])
df_manager_de = df_manager_de.explode(["referenced_tweet_id", "tweet_type"], ignore_index=True)

In [227]:
df_manager_de.head()

conversation_id   author_id                created_at  \
0  703311366262800384  1392131804 2016-02-26 20:11:11+00:00   
1  702405572012974080  1053723204 2016-02-24 08:11:53+00:00   
2  701784983829143553   554491983 2016-02-22 15:05:53+00:00   
3  701446667170615296  3365669884 2016-02-21 16:41:32+00:00   
4  700597812556386304  4858383167 2016-02-19 08:28:29+00:00   

             tweet_id lang  retweet_count  reply_count  like_count  \
0  703311366262800384   de              0            0           2   
1  702405572012974080   de              0            0           0   
2  701784983829143553   de              0            0           0   
3  701446667170615296   de              0            0           1   
4  700597812556386304   de              2            0           0   

   quote_count              source tweet_type referenced_tweet_id  \
0            0  Twitter Web Client       None                None   
1            0  Twitter Web Client       None                None   
2            0            Facebook       None                None   
3            0  Twitter Web Client       None                None   
4            0  Twitter for iPhone  retweeted  700578193334411264   

                                                text  
0  #Wirecard-Chef Markus Braun pumpt irrwitzige S...  
1  Jetzt günstig einsammeln und später absahnen? ...  
2  Morgen startet die neue Reihe R²-JazzCube! Vol...  
3  die Y-Achse des  #prusai3 nimmt Formen an :-) ...  
4  RT @aktionaer: #Wirecard: Was führt der Vorsta...

<br>
<p>Prüfen welche Datenfelder in den einzelnen Zeilen enthalten sind. War wichtig um diese anpassen zu können.</p>

In [4]:
print("conversation_id: ", df_manager_de.iloc[0]["conversation_id"], "type: ", type(df_manager_de.iloc[0]["conversation_id"]))
print("author_id: ", df_manager_de.iloc[0]["author_id"], "type: ", type(df_manager_de.iloc[0]["author_id"]))
print("created_at: ", df_manager_de.iloc[0]["created_at"], "type: ", type(df_manager_de.iloc[0]["created_at"]))
print("tweet_id: ", df_manager_de.iloc[0]["tweet_id"], "type: ", type(df_manager_de.iloc[0]["tweet_id"]))
print("lang: ", df_manager_de.iloc[0]["lang"], "type: ", type(df_manager_de.iloc[0]["lang"]))
print("retweet_count: ", df_manager_de.iloc[0]["retweet_count"], "type: ", type(df_manager_de.iloc[0]["retweet_count"]))
print("reply_count: ", df_manager_de.iloc[0]["reply_count"], "type: ", type(df_manager_de.iloc[0]["reply_count"]))
print("like_count: ", df_manager_de.iloc[0]["like_count"], "type: ", type(df_manager_de.iloc[0]["like_count"]))
print("quote_count: ", df_manager_de.iloc[0]["quote_count"], "type: ", type(df_manager_de.iloc[0]["quote_count"]))
print("source: ", df_manager_de.iloc[0]["source"], "type: ", type(df_manager_de.iloc[0]["source"]))
print("tweet_type: ", df_manager_de.iloc[0]["tweet_type"], "type: ", type(df_manager_de.iloc[0]["tweet_type"]))
print("referenced_tweet_id: ", df_manager_de.iloc[0]["referenced_tweet_id"], "type: ", type(df_manager_de.iloc[0]["referenced_tweet_id"]))
print("text: ", df_manager_de.iloc[0]["text"], "type: ", type(df_manager_de.iloc[0]["text"]))

conversation_id:  703311366262800384 type:  <class 'numpy.int64'>
author_id:  1392131804 type:  <class 'numpy.int64'>
created_at:  2016-02-26 20:11:11+00:00 type:  <class 'pandas._libs.tslibs.timestamps.Timestamp'>
tweet_id:  703311366262800384 type:  <class 'numpy.int64'>
lang:  de type:  <class 'str'>
retweet_count:  0 type:  <class 'numpy.int64'>
reply_count:  0 type:  <class 'numpy.int64'>
like_count:  2 type:  <class 'numpy.int64'>
quote_count:  0 type:  <class 'numpy.int64'>
source:  Twitter Web Client type:  <class 'str'>
tweet_type:  ['None'] type:  <class 'list'>
referenced_tweet_id:  ['None'] type:  <class 'list'>
text:  #Wirecard-Chef Markus Braun pumpt irrwitzige Summen in sein Unternehmen. Woher stammt das Geld? @SchreiberDohms  https://t.co/VdsV4nvBe4 type:  <class 'str'>


<h3>Löschen von unnötigen Informationen</h3>
<p>Hier werden Informationen aus den Tweets Texten, die nicht weiter benötigt werden. Bspw. Links</p>

In [30]:
for each in range(len(df_manager_de)):
    df_manager_de.loc[df_manager_de["text"] == df_manager_de.loc[each]["text"], "text"] = re.sub(r'http\S+', '', df_manager_de.loc[each]["text"])

<br>
<h3>Nicht Kontextbezogene Tweets löschen</h3>
<p>So kann ich prüfen ob bestimmte Begriffe in den Texten enthalten sind und mir die conversation_id ausgeben lassen. Diese lasse ich dann löschen und auch zugehörige Tweets, da diese nicht in den Kontext passen.</p>
<p>Zu löschende Begriffe</p>
<ul>
    <li>BVB</li>
    <li>Fußball</li>
    <li>Fussball</li>
    <li>Dortmund</li>
</ul>

In [228]:
tweet_id_without_context_list = []
for each in range(len(df_manager_de)):
    if ("BVB" or "Fußball" or "Fussball" or "Dortmund") in df_manager_de.iloc[each]["text"]:
        if df_manager_de.iloc[each]["tweet_id"] not in tweet_id_without_context_list:
            tweet_id_without_context_list.append(df_manager_de.iloc[each]["tweet_id"])

<p>Iteration durch die Liste mit zu löschenden Tweets, bis der Wert zu Beginn und zum Ende gleich ist und es keine
weiteren Tweets mehr gibt, die gefiltert werden müssen. </p>

In [229]:
def check_for_further_tweets():
    start_len = len(tweet_id_without_context_list)
    for each in range(len(tweet_id_without_context_list)):
        df_tweets = df_manager_de.loc[(df_manager_de["conversation_id"] == tweet_id_without_context_list[each]) | 
               (df_manager_de["referenced_tweet_id"] == str(tweet_id_without_context_list[each]))]

        for each in range(len(df_tweets)):
            if df_tweets.iloc[each]["tweet_id"] not in tweet_id_without_context_list:
                tweet_id_without_context_list.append(df_tweets.iloc[each]["tweet_id"])
                
    if start_len != len(tweet_id_without_context_list):
        check_for_further_tweets()
check_for_further_tweets()

<br>
<h4>Check</h4>
<p>Aufrufen einer Line um zu prüfen, welche Informationen enthalten sind.<br>
    Den Index abrufen um mir nur den Text in voller Länge anzeigen zu lassen, da dieser sonst gekürzt ist.</p>
   

In [69]:
df_manager_de.loc[df_manager_de["conversation_id"] == 1098841597582737409]

Empty DataFrame
Columns: [conversation_id, author_id, created_at, tweet_id, lang, retweet_count, reply_count, like_count, quote_count, source, tweet_type, referenced_tweet_id, text]
Index: []

In [49]:
df_manager_de.loc[df_manager_de.index[df_manager_de["conversation_id"] == 1278435775441494016].tolist()[0]]["text"]

'RT @RNBVB: 👨🏻\u200d⚕️ Dr. Markus #Braun ist vom Nachrichtenmagazin #Focus als "Top-Mediziner" in Deutschland ausgezeichnet worden. Zuletzt arbei…'

In [48]:
print(df_manager_de.index[df_manager_de["conversation_id"] == 1278435775441494016].tolist()[0])

14209


In [36]:
df_manager_de.loc[14209]["text"]

'RT @RNBVB: 👨🏻\u200d⚕️ Dr. Markus #Braun ist vom Nachrichtenmagazin #Focus als "Top-Mediziner" in Deutschland ausgezeichnet worden. Zuletzt arbei…'

<br>
<h3>Löschen von Retweets</h3>
<p>Alle Retweets löschen, da diese in diesem Schritt nicht benötigt werden.</p>

In [42]:
counter = 0
retweet_list = []
for each in range(len(df_manager_de)):
    if "retweeted" in df_manager_de.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)

5455


In [43]:
df_manager_de = df_manager_de.drop(retweet_list)

<h4>Dupplikate löschen</h4>
<p>Löschen von Tweets, die doppelt sind</p>

In [47]:
df_manager_de = df_manager_de.drop_duplicates(subset=["text"])